# Augmented vs Classical NSE — Comparison Notebook
This notebook compares **NMSI–π*–HDQG (augmented)** vs **vanilla NSE (classical)** using the saved `final_timeseries.npz` files.
## Instructions
1. Run two simulations with the *same* grid/nu/t_end:
   - **Augmented**: `compare_mode.classical: false`
   - **Classical**: `compare_mode.classical: true`
2. Note the output `.npz` paths (default `out_3d/` or `out_3d_long/`).
3. Set the file paths below and run the notebook.

In [ ]:
# === CONFIG: set your file paths here ===
AUG_NPZ = 'out_3d_long/final_timeseries.npz'      # augmented run (π*, γ_diss ON)
CLS_NPZ = 'out_3d_long_classic/final_timeseries.npz'  # classical run (π*, γ_diss OFF)
print('Augmented file:', AUG_NPZ)
print('Classical file:', CLS_NPZ)

In [ ]:
import numpy as np, matplotlib.pyplot as plt, os
def load_series(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f'Missing file: {path}')
    data = np.load(path)
    return {k: data[k] for k in data.files}
aug = load_series(AUG_NPZ)
cls = load_series(CLS_NPZ)
print('Augmented keys:', sorted(list(aug.keys())))
print('Classical keys:', sorted(list(cls.keys())))

In [ ]:
import numpy as np
def interp_to_common(a_t, a_y, grid):
    return np.interp(grid, a_t, a_y)
tmin = max(aug['t'].min(), cls['t'].min())
tmax = min(aug['t'].max(), cls['t'].max())
grid = np.linspace(tmin, tmax, 800)
cmp = {}
for key in ['E','Omega','Wmax']:
    cmp[f'aug_{key}'] = interp_to_common(aug['t'], aug[key], grid)
    cmp[f'cls_{key}'] = interp_to_common(cls['t'], cls[key], grid)
for key in ['eps_nu','eps_gamma','inj_pi']:
    if key in aug and key in cls:
        cmp[f'aug_{key}'] = interp_to_common(aug['t'], aug[key], grid)
        cmp[f'cls_{key}'] = interp_to_common(cls['t'], cls[key], grid)
print('Common time window:', float(tmin), '→', float(tmax))

In [ ]:
# Energy
plt.figure(); plt.plot(grid, cmp['aug_E'], label='Augmented E(t)'); plt.plot(grid, cmp['cls_E'], label='Classical E(t)', linestyle='--')
plt.xlabel('t'); plt.ylabel('Energy E(t)'); plt.title('Energy vs Time'); plt.legend(); plt.show()

In [ ]:
# Enstrophy
plt.figure(); plt.plot(grid, cmp['aug_Omega'], label='Augmented Ω(t)'); plt.plot(grid, cmp['cls_Omega'], label='Classical Ω(t)', linestyle='--')
plt.xlabel('t'); plt.ylabel('Enstrophy Ω(t)'); plt.title('Enstrophy vs Time'); plt.legend(); plt.show()

In [ ]:
# Max vorticity
plt.figure(); plt.plot(grid, cmp['aug_Wmax'], label='Augmented ||ω||∞'); plt.plot(grid, cmp['cls_Wmax'], label='Classical ||ω||∞', linestyle='--')
plt.xlabel('t'); plt.ylabel('||ω||∞'); plt.title('Max Vorticity vs Time'); plt.legend(); plt.show()

In [ ]:
# Budget terms (if present)
def maybe_plot(key, ylabel, title):
    K1=f'aug_{key}'; K2=f'cls_{key}'
    if K1 in cmp and K2 in cmp:
        plt.figure(); plt.plot(grid, cmp[K1], label=f'Augmented {key}'); plt.plot(grid, cmp[K2], label=f'Classical {key}', linestyle='--')
        plt.xlabel('t'); plt.ylabel(ylabel); plt.title(title); plt.legend(); plt.show()
    else:
        print('Skipping', key, '(not present in both)')
maybe_plot('eps_nu','ε_ν','Viscous Dissipation ε_ν')
maybe_plot('eps_gamma','ε_γ','Intermittent Dissipation ε_γ')
maybe_plot('inj_pi','Injection','π* Energy Injection')

In [ ]:
# Summary metrics
def summarize(prefix):
    return {
        'E_mean': float(np.mean(cmp[f'{prefix}_E'])),
        'E_max': float(np.max(cmp[f'{prefix}_E'])),
        'Omega_mean': float(np.mean(cmp[f'{prefix}_Omega'])),
        'Omega_max': float(np.max(cmp[f'{prefix}_Omega'])),
        'Wmax_mean': float(np.mean(cmp[f'{prefix}_Wmax'])),
        'Wmax_max': float(np.max(cmp[f'{prefix}_Wmax']))
    }
aug_sum = summarize('aug'); cls_sum = summarize('cls'); aug_sum, cls_sum

In [ ]:
# Export CSV
import csv
rows = [
 ['metric','augmented','classical'],
 ['E_mean', aug_sum['E_mean'], cls_sum['E_mean']],
 ['E_max',  aug_sum['E_max'],  cls_sum['E_max']],
 ['Omega_mean', aug_sum['Omega_mean'], cls_sum['Omega_mean']],
 ['Omega_max',  aug_sum['Omega_max'],  cls_sum['Omega_max']],
 ['Wmax_mean', aug_sum['Wmax_mean'], cls_sum['Wmax_mean']],
 ['Wmax_max',  aug_sum['Wmax_max'],  cls_sum['Wmax_max']],
]
out_csv = 'compare_summary.csv'
with open(out_csv, 'w', newline='') as f:
    w = csv.writer(f); w.writerows(rows)
print('Saved', out_csv)